<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_fss/blob/main/PIFWIA_FSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FSS - Fish School Search ou Pesquisa de Cardumes de Peixes é um método proposto pelos pesquisadores Fernando Buarque e Carmelo Filho em 2008 para resolver problemas de otimização inspirado no comportamento de cardumes, onde os mecanismos de alimentação e movimento coordenados dos peixes foram usados ​​como inspiração para criar os operadores de busca, tendo como ideia central a de fazer os peixes nadarem em direção ao gradiente positivo para se alimentar e ganhar peso, dessa forma os peixes mais pesados ​​têm mais influência no processo de busca como um todo, o que faz com que o baricentro do cardume se desloque para lugares melhores no espaço de busca ao longo das iterações, onde todos os peixes realizam buscas locais e o cardume agrega informações sociais.

In [ ]:
#instalação da biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#bibliotecas
import random
import math
import matplotlib.pyplot as plt
import Orange

In [ ]:
#hiperparâmetros
DIMENSOES = 6 #determina a quantidade de dimensões do problema
ITERACOES = 200 #quantiddade máxima de ciclos (episódios) especificando quantas explorações podem ser realizadas
CARDUME = 100 #tamanho da população correspondente ao número de peixes no aquário (tamanho do cardume)
E1 = -100 #extremos 1 e 3
E2 = 100 #extremos 2 e 4
LIMITES = [E1, E2] #(bound) determina os valores maximos e minimos do espaço de busca (tamenho do aquário)
FCUSTO = 'parábola' #(fitness) função custo que pode ser uma esfera ou parábola (rosenbrock) ou outra função custo que defina a densidade de alimentação dos peixes no aquário
PASSO_INICIAL = 0.01 #(swimming inicial) conduz os movimentos dos peixes
PASSO_FINAL = 0.000001 #(swimming final) conduz os movimentos dos peixes
PEIXES = [] #(swarm) array da criação dos peixes

In [ ]:
#importação dos dados
from google.colab import files  
files.upload()

In [ ]:
#criação do objeto de dados com base no caminho gerado com a importação do arquivo
dados = Orange.data.Table("/content/dados.csv")

In [ ]:
#exploração dos metadados da base de dados importada
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)
#exploração dos domínios dos atributos (campos da base de dados)
dados.domain.attributes
#exploração dos dados (quantidade de campos e registros da base de dados)
print("Qtde de Registros:", len(dados))
#exibição dos 5 primeiros registros para análise dos dados importados
i = 0
for d in dados[:5]:
  i += 1
  print(i, d)

In [ ]:
#criação dos arrays das dimensões do problema
#complexidade
#1 = muito baixa complexidade
#2 = baixa complexidade
#3 = média complexidade
#4 = alta complexidade
#5 = muito alta complexidade
periodo = []
complexidade = []
pagina = []
prazo = []
revisao = []
entrega = []
for d in dados[:CARDUME]:
  periodo.append(d[1])
  complexidade.append(d[2])
  pagina.append(d[3])
  prazo.append(d[4])
  revisao.append(d[5])
  entrega.append(d[6])
for i in range(CARDUME):
  print("id:",i+1,"período:",periodo[i],"complexidade:",complexidade[i],"página:",pagina[i],"prazo:",prazo[i],"revisões:",revisao[i],"entrega:",entrega[i])

In [ ]:
#peixe - unidade base da otimização, posicionado numa determinada posição no espaço de busca do problema, representando uma possível solução para o problema
class Peixe:
  def __init__(self):
    self.posicao = [0.0] * DIMENSOES
    self.fitness = 0.0
    self.dif_distancia = [0.0] * DIMENSOES
    self.dif_fitness = 0.0
    self.peso = 1.0

In [ ]:
#função fitness ou objetivo ou otimização - usada para verificação se o mínimo global está dentro de uma função custo determinada pelo argumento
def custo(posicao): 
  return sum([coord ** 2 for coord in posicao])
def fitness(problema, posicoes):
  total = 0.0
  if problema == 'esfera':
    for i in range(DIMENSOES):
      total += posicoes[i]**2
  elif problema == 'parábola':
    for i in range(DIMENSOES-1):
      total += 100*(posicoes[i+1] - posicoes[i]**2)**2 + (1-posicoes[i])**2 #rosenbrok
  elif problema == 'custo':
    for i in range(DIMENSOES-1):
      total += 1 / abs(custo(posicoes))
  else:
    print('Problema não encontrado!')
  return total

In [ ]:
#inicialização do população (cardume)
def inicializar_fss(problema):
  peixes = []
  for i in range(CARDUME):
    peixes.append(Peixe())
    posicoes = []
    for j in range(DIMENSOES):
      posicao = random.random() * (LIMITES[1] - LIMITES[0]) - LIMITES[1] 
      posicoes.append(posicao)
    peixes[-1].posicao = posicoes
    peixes[-1].fitness = fitness(problema, peixes[-1].posicao)
  return peixes

In [ ]:
#posições iniciais dos peixes
print("Plano Cartesiano")
E1 = -100
E2 = 100
plt.axis([E1, E2, E1, E2])
plt.plot(0,0, marker='*', markersize=10, color='b')
PEIXES = inicializar_fss(FCUSTO)
for i in range (CARDUME):
  p = PEIXES[i]
  #variáveis das DIMENSÕES do problema
  d1,d2,d3,d4,d5,d6 = zip(p.posicao)
  plt.plot(d1,d2,d3,d4,d5,d6, marker='o')
plt.show()

In [ ]:
#movimento individual - cada peixe escolhe aleatoriamente uma nova posição em sua vizinhança e a avalia com a função aptidão ou objetivo (fitness)
def movimento_individual(populacao, passo_atual, problema):
  for peixe in populacao:
    nova_posicao = []
    novo_fitness = 0.0
    for dimensao in range(DIMENSOES):
      #calcula a nova posicao
      direcao = random.random()*2 - 1
      mudanca = direcao*passo_atual*(LIMITES[1] - LIMITES[0])
      nova_posicao.append( peixe.posicao[dimensao] + mudanca )
      #verifica se ultrapassa os limites superior e inferior
      if nova_posicao[-1] > LIMITES[1]:
        nova_posicao[-1] = LIMITES[1]
      elif nova_posicao[-1] < LIMITES[0]:
        nova_posicao[-1] = LIMITES[0]         
    #verifica o fitness da nova posicao
    novo_fitness = fitness(problema, nova_posicao)
    #o peixe escolhe a melhor posicao e salva a diferenca de fitness e de posicao
    if(novo_fitness < peixe.fitness):
      peixe.dif_fitness = peixe.fitness - novo_fitness
      peixe.fitness = novo_fitness
      for dim in range(DIMENSOES):
        peixe.dif_distancia[dim] = nova_posicao[dim] - peixe.posicao[dim]
        peixe.posicao[dim] = nova_posicao[dim]
    else:
      peixe.dif_fitness = 0
      for dim in range(DIMENSOES):
        peixe.dif_distancia[dim] = 0

In [ ]:
#operador de alimentação - cada peixe pode aumentar seu peso dependendo da taxa de sucesso obtida pelo movimento indiviaul (feeding)
def alimentacao(populacao):
  maior_diferenca_fitness = 0
  mudanca_peso = 0
  for peixe in populacao:
    mudanca_peso += peixe.peso
    if peixe.dif_fitness > maior_diferenca_fitness:
      maior_diferenca_fitness = peixe.dif_fitness
  if maior_diferenca_fitness != 0:
    for peixe in populacao:
      peixe.peso += peixe.dif_fitness/maior_diferenca_fitness
      mudanca_peso -= peixe.peso
    return mudanca_peso
  else:
    return 0

In [ ]:
#movimento instintivo - somente peixes que realizaram movimentos individuais com sucesso poderão determinar a direção do movimento do cardume (swimming)
def movimento_instintivo(populacao):
  total_diferenca_fitness = 0.0
  vetor_i = [0.0] * DIMENSOES
  for peixe in populacao:
    total_diferenca_fitness += peixe.dif_fitness
    for i in range(DIMENSOES):
      vetor_i[i] += peixe.dif_distancia[i]*peixe.dif_fitness
  if total_diferenca_fitness != 0:
    for i in range(DIMENSOES):
      vetor_i[i] /= total_diferenca_fitness
    for peixe in populacao:
      for i in range(DIMENSOES):
        peixe.posicao[i] += vetor_i[i]
        #verifica se ultrapassa os limites superior e inferior
        if peixe.posicao[i] > LIMITES[1]:
          peixe.posicao[i] = LIMITES[1]
        elif peixe.posicao[i] < LIMITES[0]:
          peixe.posicao[i] = LIMITES[0]

In [ ]:
#calculo do baricentro - ponto de aplicação da força de gravidade de acordo com a força-peso (memória do cardume)
def calcular_baricentro(populacao):
  baricentro = [0.0] * DIMENSOES
  peso_total = 0.0
  for peixe in populacao:
    peso_total += peixe.peso
    for i in range(DIMENSOES):
      baricentro[i] += peixe.peso * peixe.posicao[i]
  for i in range(DIMENSOES):
    baricentro[i] /= peso_total
  return baricentro

In [ ]:
#calculo da distancia
def distancia(lista_1, lista_2):
  dist = 0.0
  for i in range(len(lista_1)):
    dist += (lista_1[i] - lista_2[i])**2
  return math.sqrt(dist)

In [ ]:
#movimento volitivo - poder de escolha baseado na taxa geral de sucesso do cardume (swimming)
def movimento_volitivo(populacao, baricentro, mudanca_peso, passo):
  for peixe in populacao:
    for i in range(DIMENSOES):
      direcao = random.random()
      if mudanca_peso < 0:
        peixe.posicao[i] -= 2*passo*direcao*(LIMITES[1] - LIMITES[0])* ((peixe.posicao[i] - baricentro[i])/distancia(peixe.posicao, baricentro))
      else:
        peixe.posicao[i] += 2*passo*direcao*(LIMITES[1] - LIMITES[0])* ((peixe.posicao[i] - baricentro[i])/distancia(peixe.posicao, baricentro))
      #verifica se ultrapassa os limites superior e inferior
      if peixe.posicao[i] > LIMITES[1]:
        peixe.posicao[i] = LIMITES[1]
      elif peixe.posicao[i] < LIMITES[0]:
        peixe.posicao[i] = LIMITES[0]

In [ ]:
#atualização do passo - conduz os movimentos dos peixes
def atualizar_passo(passo):
    return passo - ((PASSO_INICIAL - PASSO_FINAL) / ITERACOES)

In [ ]:
#otimizador FSS
passo = PASSO_INICIAL
melhor_fitness = 999999999999
fitness_tempo = []
populacao = PEIXES
#print("Melhor fitness inicial: ", melhor_fitness)
for i in range(ITERACOES):
  print("Iteração: {:.0f}".format(i+1))
  movimento_individual(populacao, passo, FCUSTO)
  mudanca_peso = alimentacao(populacao)
  movimento_instintivo(populacao)
  baricentro = calcular_baricentro(populacao)
  movimento_volitivo(populacao, baricentro, mudanca_peso, passo)
  passo = atualizar_passo(passo)
  #plota os eixos
  plt.axis([E1, E2, E1, E2])
  #ponto ideal
  plt.plot(0,0, marker='*', markersize=10, color='b')
  for i in range(CARDUME):
    populacao[i].fitness = fitness(FCUSTO, populacao[i].posicao)
  for i in populacao:
    if melhor_fitness > i.fitness:
      melhor_fitness = i.fitness
  fitness_tempo.append(melhor_fitness)
  #exibe o enxame - mostrando no gráfico as posições atuais de cada paixe no aquário
  for i in range (CARDUME):
    p = populacao[i]
    #variáveis das DIMENSÕES do problema
    d1,d2,d3,d4,d5,d6 = zip(p.posicao)
    plt.plot(d1,d2,d3,d4,d5,d6, marker='o')
  #ponto gbest
  plt.plot(melhor_fitness, marker='x', markersize=15, color='r')
  plt.show()
  print('Melhor Ponto:',melhor_fitness)
  print("")
#print("Melhor fitness final: ", melhor_fitness)

In [ ]:
#resultado
x = []
y = []
for i in range(len(fitness_tempo)):
  x.append(i)
  y.append(fitness_tempo[i])
plt.plot(x, y)
plt.title("Curva de Convergência do FSS")
plt.xlabel("Iteração")
plt.ylabel("Melhor Ponto")
plt.tight_layout()